# Folder RAG - Interactive Demo

This notebook demonstrates the core functionality of the Folder RAG project:
- Database operations
- Japanese tokenization with MeCab
- Full-text search with FTS5
- Configuration management

**Phase 1 Complete** ✓

Run each cell in order to explore the features!

## 1. Setup - Import Modules

In [2]:
import sys
sys.path.insert(0, '.')

from src.core.database import Database
from src.core.tokenizer import get_tokenizer
from src.core.config import Config
from src.core.models import Document, Chunk, DocumentStatus
from datetime import datetime
import uuid

print("✓ All modules imported successfully!")
print(f"✓ Python version: {sys.version}")

✓ All modules imported successfully!
✓ Python version: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]


## 2. Test Japanese Tokenizer

MeCab automatically splits Japanese text into words for better search.

In [3]:
tokenizer = get_tokenizer()

# Check if MeCab is available
if tokenizer.mecab:
    print("✓ MeCab is available and initialized\n")
else:
    print("✗ MeCab not available, using fallback\n")

# Test Japanese tokenization
test_texts = [
    "機械学習はPythonで実装できます",
    "自然言語処理とRAGアプリケーション",
    "深層学習、ニューラルネットワーク、AI技術",
    "Python is a programming language",
    "日本語とEnglishの混在テキスト"
]

print("=" * 70)
for text in test_texts:
    print(f"Original:  {text}")
    tokenized = tokenizer.tokenize(text)
    print(f"Tokenized: {tokenized}")
    tokens = tokenizer.get_tokens_list(text)
    print(f"Count:     {len(tokens)} tokens")
    print("-" * 70)

✓ MeCab is available and initialized

Original:  機械学習はPythonで実装できます
Tokenized: 機械 学習 は Python で 実装 でき ます
Count:     8 tokens
----------------------------------------------------------------------
Original:  自然言語処理とRAGアプリケーション
Tokenized: 自然 言語 処理 と RAG アプリケーション
Count:     6 tokens
----------------------------------------------------------------------
Original:  深層学習、ニューラルネットワーク、AI技術
Tokenized: 深層 学習 、 ニューラル ネットワーク 、 AI 技術
Count:     8 tokens
----------------------------------------------------------------------
Original:  Python is a programming language
Tokenized: Python is a programming language
Count:     5 tokens
----------------------------------------------------------------------
Original:  日本語とEnglishの混在テキスト
Tokenized: 日本 語 と English の 混在 テキスト
Count:     7 tokens
----------------------------------------------------------------------


## 3. Initialize Database

Create a database instance and verify the schema.

In [5]:
# Create database (uses data/folderrag.db)
db = Database()

print("✓ Database initialized")
print(f"✓ Database path: {db.db_path}")
print(f"✓ Current chunk count: {db.get_chunk_count()}")
print(f"✓ Current document count: {len(db.get_all_documents())}")

✓ Database initialized
✓ Database path: data\folderrag.db
✓ Current chunk count: 0
✓ Current document count: 0


## 4. Add Test Documents

Let's add some sample documents with Japanese and English text.

In [8]:
# Create test documents
doc1 = Document(
    id=str(uuid.uuid4()),
    path="C:\\Development\\projects\\myRAG\\test.pdf",
    title="ハルシネーション.pdf",
    ext=".pdf",
    mtime=datetime.now(),
    size=2048,
    status=DocumentStatus.INDEXED
)

# doc2 = Document(
#     id=str(uuid.uuid4()),
#     path="C:/test/python_guide.pdf",
#     title="Python Programming Guide.pdf",
#     ext=".pdf",
#     mtime=datetime.now(),
#     size=3072,
#     status=DocumentStatus.INDEXED
# )

# Add to database
db.add_document(doc1)
# db.add_document(doc2)

print("✓ Added 1 test documents:")
print(f"  1. {doc1.title}")
# print(f"  2. {doc2.title}")

✓ Added 1 test documents:
  1. ハルシネーション.pdf


## 5. Add Text Chunks

Add text chunks in Japanese and English. Note how Japanese text is automatically tokenized!

In [9]:
# Japanese chunks for doc1
japanese_chunks = [
    "機械学習は人工知能の一分野です。データからパターンを学習します。",
    "教師あり学習、教師なし学習、強化学習の三つの主要な学習方法があります。",
    "Pythonは機械学習で最も人気のあるプログラミング言語です。",
    "深層学習はニューラルネットワークを使用した機械学習の手法です。",
]

# English chunks for doc2
english_chunks = [
    "Python is a high-level programming language known for its simplicity.",
    "Machine learning libraries like scikit-learn and TensorFlow are popular.",
    "Python supports multiple programming paradigms including OOP and functional.",
    "Data science and artificial intelligence applications often use Python.",
]

# Add Japanese chunks
print("Adding Japanese chunks...")
for i, text in enumerate(japanese_chunks):
    chunk = Chunk(
        id=str(uuid.uuid4()),
        document_id=doc1.id,
        page=i + 1,
        start_offset=i * 100,
        end_offset=(i + 1) * 100,
        text=text,
        text_hash=f"hash_jp_{i}"
    )
    db.add_chunk(chunk)
    print(f"  ✓ Chunk {i+1}: {text[:30]}...")

# Add English chunks
print("\nAdding English chunks...")
for i, text in enumerate(english_chunks):
    chunk = Chunk(
        id=str(uuid.uuid4()),
        document_id=doc2.id,
        page=i + 1,
        start_offset=i * 100,
        end_offset=(i + 1) * 100,
        text=text,
        text_hash=f"hash_en_{i}"
    )
    db.add_chunk(chunk)
    print(f"  ✓ Chunk {i+1}: {text[:50]}...")

print(f"\n✓ Total chunks in database: {db.get_chunk_count()}")

Adding Japanese chunks...
  ✓ Chunk 1: 機械学習は人工知能の一分野です。データからパターンを学習しま...
  ✓ Chunk 2: 教師あり学習、教師なし学習、強化学習の三つの主要な学習方法が...
  ✓ Chunk 3: Pythonは機械学習で最も人気のあるプログラミング言語です...
  ✓ Chunk 4: 深層学習はニューラルネットワークを使用した機械学習の手法です...

Adding English chunks...


NameError: name 'doc2' is not defined

## 6. Search Test - Japanese Keywords

Now let's test FTS5 search with Japanese keywords. Thanks to MeCab, we can search for individual words!

In [10]:
def search_and_display(query, limit=5):
    """Helper function to search and display results."""
    print(f"\n{'='*70}")
    print(f"Search Query: '{query}'")
    print(f"{'='*70}")
    
    results = db.search_chunks_fts(query, limit=limit)
    
    if not results:
        print("No results found.")
        return
    
    print(f"Found {len(results)} results:\n")
    
    for i, (chunk_id, score) in enumerate(results, 1):
        chunk = db.get_chunk(chunk_id)
        doc = db.get_document(chunk.document_id)
        print(f"{i}. Score: {score:.4f}")
        print(f"   Document: {doc.title}")
        print(f"   Page: {chunk.page}")
        print(f"   Text: {chunk.text}")
        print()

# Test Japanese searches
search_and_display("学習")
search_and_display("教師なし")
search_and_display("Python")



Search Query: '学習'
Found 4 results:

1. Score: -0.0000
   Document: ハルシネーション.pdf
   Page: 2
   Text: 教師 あり 学習 、 教師 なし 学習 、 強化 学習 の 三 つ の 主要 な 学習 方法 が あり ます 。

2. Score: -0.0000
   Document: ハルシネーション.pdf
   Page: 4
   Text: 深層 学習 は ニューラル ネットワーク を 使用 し た 機械 学習 の 手法 です 。

3. Score: -0.0000
   Document: ハルシネーション.pdf
   Page: 1
   Text: 機械 学習 は 人工 知能 の 一 分野 です 。 データ から パターン を 学習 し ます 。

4. Score: -0.0000
   Document: ハルシネーション.pdf
   Page: 3
   Text: Python は 機械 学習 で 最も 人気 の ある プログラミング 言語 です 。


Search Query: '教師なし'
Found 1 results:

1. Score: -1.8595
   Document: ハルシネーション.pdf
   Page: 2
   Text: 教師 あり 学習 、 教師 なし 学習 、 強化 学習 の 三 つ の 主要 な 学習 方法 が あり ます 。


Search Query: 'Python'
Found 1 results:

1. Score: -0.9282
   Document: ハルシネーション.pdf
   Page: 3
   Text: Python は 機械 学習 で 最も 人気 の ある プログラミング 言語 です 。



## 7. Search Test - English Keywords

In [11]:
search_and_display("programming")
search_and_display("scikit-learn")
search_and_display("artificial intelligence")


Search Query: 'programming'
No results found.

Search Query: 'scikit-learn'


OperationalError: no such column: learn

## 8. Configuration Management

View and modify application settings.

In [ ]:
config = Config(db)
settings = config.get_settings()

print("Current Settings:")
print(f"  Chunk size: {settings.chunk_size} tokens")
print(f"  Chunk overlap: {settings.chunk_overlap} tokens")
print(f"  Top K results: {settings.top_k}")
print(f"  Allowed extensions: {', '.join(settings.allowed_ext)}")
print(f"  Embedding model: {settings.embedding_model}")
print(f"  Generation mode: {settings.generation_mode.value}")
print(f"  Included paths: {settings.included_paths if settings.included_paths else 'None'}")

## 9. Summary & Statistics

Let's see what we've created!

In [ ]:
all_docs = db.get_all_documents()
total_chunks = db.get_chunk_count()

print("=" * 70)
print("DATABASE STATISTICS")
print("=" * 70)
print(f"\nTotal Documents: {len(all_docs)}")
print(f"Total Chunks: {total_chunks}")
print(f"\nDocuments:")
for i, doc in enumerate(all_docs, 1):
    chunks = db.get_chunks_by_document(doc.id)
    print(f"  {i}. {doc.title}")
    print(f"     - Status: {doc.status.value}")
    print(f"     - Chunks: {len(chunks)}")
    print(f"     - Size: {doc.size} bytes")

print("\n" + "=" * 70)
print("✓ Phase 1 Demo Complete!")
print("=" * 70)
print("\nWhat's working:")
print("  ✓ Japanese tokenization with MeCab")
print("  ✓ Full-text search with FTS5")
print("  ✓ Database operations")
print("  ✓ Configuration management")
print("\nComing in Phase 2:")
print("  • Automatic PDF/TXT file indexing")
print("  • Folder scanning")
print("  • Progress tracking")
print("=" * 70)

# Phase 2: File Indexing Pipeline

Now let's demonstrate the complete indexing pipeline that was built in Phase 2!

## 1. File Ingestion - Scan Folders for Documents

The ingestion module scans folders and finds all PDF, TXT, and MD files.

In [12]:
from src.indexing.ingestion import Ingestion

# Create ingestion with our database
ingestion = Ingestion(db)

# Scan the test data folder
print("📁 Scanning test data folder...")
added, updated, errors = ingestion.scan_and_add('tests/test_data', recursive=False)

print(f"\n✅ Added: {added} files")
print(f"🔄 Updated: {updated} files")
print(f"❌ Errors: {len(errors)}")

if errors:
    for error in errors:
        print(f"  - {error}")

# Show what was found
print(f"\n📋 Pending documents:")
pending = ingestion.get_pending_documents()
for doc in pending:
    print(f"  - {doc.title} ({doc.ext}) - {doc.size} bytes")

📁 Scanning test data folder...

✅ Added: 3 files
🔄 Updated: 0 files
❌ Errors: 0

📋 Pending documents:
  - sample.md (.md) - 686 bytes
  - sample.pdf (.pdf) - 1910 bytes
  - sample.txt (.txt) - 613 bytes


## 2. Text Extraction - Extract from PDF, TXT, MD

Let's extract text from one of the documents to see how it works.

In [13]:
from src.indexing.extractor import Extractor

extractor = Extractor()

# Extract from the PDF file
if pending:
    # Get the first pending document
    test_doc = pending[0]
    print(f"📄 Extracting text from: {test_doc.title}\n")
    
    extracted = extractor.extract(test_doc.path)
    
    print(f"📊 Extraction Results:")
    print(f"  - Total pages: {len(extracted.pages)}")
    print(f"  - Total characters: {extracted.total_chars}")
    
    print(f"\n📖 Page-by-page content:")
    for page in extracted.pages:
        print(f"\n  Page {page.page_number} ({page.char_count} chars):")
        # Show first 150 characters of each page
        preview = page.text[:150].replace('\n', ' ')
        print(f"    {preview}...")
        
    # Show full text of first page
    print(f"\n📝 Full text of page 1:")
    print(extracted.pages[0].text)

📄 Extracting text from: sample.md

📊 Extraction Results:
  - Total pages: 1
  - Total characters: 549

📖 Page-by-page content:

  Page 1 (549 chars):
    # Test Markdown File  This is a test markdown file for myRAG.  ## Section 1: Introduction  This document contains **formatted text** with _italics_ an...

📝 Full text of page 1:
# Test Markdown File

This is a test markdown file for myRAG.

## Section 1: Introduction

This document contains **formatted text** with _italics_ and other markdown features.

## Section 2: Japanese Content

日本語のコンテンツもテストします。

### Subsection 2.1

機械学習とは、コンピュータがデータから学習するアルゴリズムです。

## Section 3: Code Example

```python
def hello_world():
    print("Hello, RAG!")
```

## Section 4: Lists

- Item 1
- Item 2
- Item 3

### Ordered List

1. First item
2. Second item
3. Third item

## Conclusion

This markdown file tests various formatting features.


## 3. Text Chunking - Split into Searchable Segments

Now let's chunk the extracted text with overlap for better search context.

In [14]:
from src.indexing.chunker import Chunker

# Create chunker with smaller chunks for demo
chunker = Chunker(chunk_size=100, chunk_overlap=20)

# Chunk the extracted text
if extracted:
    pages_data = [(page.page_number, page.text) for page in extracted.pages]
    chunks = chunker.chunk_pages(pages_data)
    
    print(f"✂️  Chunking Results:")
    print(f"  - Total chunks: {len(chunks)}")
    print(f"  - Chunk size: {chunker.chunk_size} tokens")
    print(f"  - Overlap: {chunker.chunk_overlap} tokens")
    
    print(f"\n📦 Chunk Details:")
    for i, chunk in enumerate(chunks[:5], 1):  # Show first 5 chunks
        print(f"\n  Chunk {i}:")
        print(f"    Page: {chunk.page_number}")
        print(f"    Tokens: {chunk.token_count}")
        print(f"    Hash: {chunk.text_hash[:16]}...")
        print(f"    Text: {chunk.text[:100]}...")
    
    if len(chunks) > 5:
        print(f"\n  ... and {len(chunks) - 5} more chunks")

✂️  Chunking Results:
  - Total chunks: 7
  - Chunk size: 100 tokens
  - Overlap: 20 tokens

📦 Chunk Details:

  Chunk 1:
    Page: 1
    Tokens: 100
    Hash: 8b82db94dd0ba2ba...
    Text: #   T e s t   M a r k d o w n   F i l e   T h i s   i s   a   t e s t   m a r k d o w n   f i l e   ...

  Chunk 2:
    Page: 1
    Tokens: 100
    Hash: e9a1106dc7b3bd81...
    Text: t r o d u c t i o n   T h i s   d o c u m e n t   c o n t a i n s   *   *   f o r m a t t e d   t e ...

  Chunk 3:
    Page: 1
    Tokens: 100
    Hash: d3adff138c5750da...
    Text: s   .   #   #   S e c t i o n   2   :   J a p a n e s e   C o n t e n t   日 本   語   の   コ ン テ ン ツ   ...

  Chunk 4:
    Page: 1
    Tokens: 100
    Hash: 2cd8f9e05010f33e...
    Text: ピ ュ ー タ   が   デ ー タ   か ら   学 習   す る   ア ル ゴ リ ズ ム   で す   。   #   #   S e c t i o n   3   :   C o ...

  Chunk 5:
    Page: 1
    Tokens: 100
    Hash: ec7921fba9ae7c63...
    Text: t   ( "   H e l l o   ,   R A G   ! " )   ` ` `   #   #   S e c t i o n   

## 4. Complete Indexing Pipeline - End-to-End

Let's process all pending documents through the complete pipeline!

In [15]:
from src.core.models import Chunk

# Use standard chunk size for indexing
production_chunker = Chunker(chunk_size=800, chunk_overlap=150)

print("🚀 Starting indexing pipeline...\n")

total_chunks_added = 0

for doc in pending:
    print(f"📄 Processing: {doc.title}")
    
    try:
        # Step 1: Extract text
        extracted = extractor.extract(doc.path)
        print(f"  ✓ Extracted {len(extracted.pages)} pages, {extracted.total_chars} chars")
        
        # Step 2: Chunk text
        pages_data = [(page.page_number, page.text) for page in extracted.pages]
        chunks = production_chunker.chunk_pages(pages_data)
        print(f"  ✓ Created {len(chunks)} chunks")
        
        # Step 3: Add chunks to database
        for chunk in chunks:
            chunk_obj = Chunk(
                id=None,  # Auto-generated
                document_id=doc.id,
                page=chunk.page_number,
                start_offset=chunk.start_offset,
                end_offset=chunk.end_offset,
                text=chunk.text,
                text_hash=chunk.text_hash
            )
            db.add_chunk(chunk_obj)
            total_chunks_added += 1
        
        # Step 4: Mark as indexed
        db.update_document_status(doc.id, DocumentStatus.INDEXED)
        print(f"  ✓ Status: INDEXED\n")
        
    except Exception as e:
        db.update_document_status(doc.id, DocumentStatus.ERROR, str(e))
        print(f"  ✗ Error: {str(e)}\n")

print(f"✅ Pipeline complete! Added {total_chunks_added} chunks to database")

🚀 Starting indexing pipeline...

📄 Processing: sample.md
  ✓ Extracted 1 pages, 549 chars
  ✓ Created 1 chunks
  ✓ Status: INDEXED

📄 Processing: sample.pdf
  ✓ Extracted 3 pages, 424 chars
  ✓ Created 3 chunks
  ✓ Status: INDEXED

📄 Processing: sample.txt
  ✓ Extracted 1 pages, 486 chars
  ✓ Created 1 chunks
  ✓ Status: INDEXED

✅ Pipeline complete! Added 5 chunks to database


## 5. Verify Indexed Documents

Let's check what's now in our database!

In [16]:
print("📊 Database Statistics:\n")

# Get all documents
all_documents = db.get_all_documents()
indexed_docs = [d for d in all_documents if d.status == DocumentStatus.INDEXED]

print(f"Total documents: {len(all_documents)}")
print(f"Indexed documents: {len(indexed_docs)}")

print(f"\n📋 Indexed Documents:")
for doc in indexed_docs:
    doc_chunks = db.get_chunks_by_document(doc.id)
    print(f"\n  {doc.title}")
    print(f"    Type: {doc.ext}")
    print(f"    Size: {doc.size:,} bytes")
    print(f"    Status: {doc.status.value}")
    print(f"    Chunks: {len(doc_chunks)}")
    
    # Show a sample chunk
    if doc_chunks:
        sample = doc_chunks[0]
        print(f"    Sample chunk (page {sample.page}):")
        print(f"      {sample.text[:100]}...")

# Total chunk count
total_chunks = db.get_chunk_count()
print(f"\n📦 Total chunks in database: {total_chunks}")

📊 Database Statistics:

Total documents: 4
Indexed documents: 4

📋 Indexed Documents:

  ハルシネーション.pdf
    Type: .pdf
    Size: 2,048 bytes
    Status: indexed
    Chunks: 4
    Sample chunk (page 1):
      機械 学習 は 人工 知能 の 一 分野 です 。 データ から パターン を 学習 し ます 。...

  sample.md
    Type: .md
    Size: 686 bytes
    Status: indexed
    Chunks: 1
    Sample chunk (page 1):
      # T e s t M a r k d o w n F i l e T h i s i s a t e s t m a r k d o w n f i l e f o r m y R A G . # ...

  sample.pdf
    Type: .pdf
    Size: 1,910 bytes
    Status: indexed
    Chunks: 3
    Sample chunk (page 1):
      T e s t   P D F   D o c u m e n t 
 P a g e   1 
 T h i s   i s   a   t e s t   P D F   f i l e   f ...

  sample.txt
    Type: .txt
    Size: 613 bytes
    Status: indexed
    Chunks: 1
    Sample chunk (page 1):
      T h i s i s a t e s t t e x t f i l e f o r t h e m y R A G i n d e x i n g s y s t e m . T h i s f ...

📦 Total chunks in database: 9


## 6. Search Testing - Ready for Phase 3!

The chunks are now indexed and ready to be searched. Phase 3 will add keyword + semantic search!

In [17]:
print("🔍 Basic FTS5 Search Test:\n")

# Try searching for common words
test_queries = ["markdown", "python", "file", "document"]

for query in test_queries:
    results = db.search_chunks_fts(query, limit=3)
    print(f"Query: '{query}' → {len(results)} results")
    
    for i, result in enumerate(results[:2], 1):
        chunk = result.chunk
        print(f"  {i}. Page {chunk.page}: {chunk.text[:60]}...")

print(f"\n💡 Note: FTS5 is working! Phase 3 will add:")
print("  - Proper query tokenization")
print("  - Semantic search with embeddings")
print("  - Hybrid search (keyword + semantic)")
print("  - Re-ranking of results")
print("  - RAG answer generation with citations")

🔍 Basic FTS5 Search Test:

Query: 'markdown' → 0 results
Query: 'python' → 1 results


AttributeError: 'tuple' object has no attribute 'chunk'

---

## Phase 2 Summary

You've just seen the complete file indexing pipeline:

1. **Ingestion**: Scanned folders and found 3 documents (PDF, TXT, MD)
2. **Extraction**: Extracted text with page numbers preserved
3. **Chunking**: Split text into overlapping chunks with Japanese tokenization
4. **Storage**: Added chunks to database with automatic FTS5 indexing
5. **Verification**: All documents successfully indexed

**Test Coverage**: 69 tests passing (33 Phase 1 + 36 Phase 2)

**Next**: Phase 3 will implement search and RAG answer generation!